In [9]:
import numpy as np 
import bagpipes as pipes

from astropy.io import fits

#load the filter curves
def load_goodss(ID):
    """ Load CANDELS GOODS South photometry from the Guo et al. (2013) catalogue. """

    # load up the relevant columns from the catalogue.
    cat = np.loadtxt("hlsp_candels_hst_wfc3_goodss-tot-multiband_f160w_v1-1photom_cat.txt",
                     usecols=(10, 13, 16, 19, 25, 28, 31, 34, 37, 40, 43, 46, 49, 52, 55,
                              11, 14, 17, 20, 26, 29, 32, 35, 38, 41, 44, 47, 50, 53, 56))
    
    # Find the correct row for the object we want.
    row = int(ID) - 1

    # Extract the object we want from the catalogue.
    fluxes = cat[row, :15]
    fluxerrs = cat[row, 15:]

    # Turn these into a 2D array.
    photometry = np.c_[fluxes, fluxerrs]

    # blow up the errors associated with any missing fluxes.
    for i in range(len(photometry)):
        if (photometry[i, 0] == 0.) or (photometry[i, 1] <= 0):
            photometry[i,:] = [0., 9.9*10**99.]
            
    # Enforce a maximum SNR of 20, or 10 in the IRAC channels.
    for i in range(len(photometry)):
        if i < 10:
            max_snr = 20.
            
        else:
            max_snr = 10.
        
        if photometry[i, 0]/photometry[i, 1] > max_snr:
            photometry[i, 1] = photometry[i, 0]/max_snr

    return photometry

goodss_filt_list = np.loadtxt("filters1/goodss_filt_list.txt", dtype="str")

#dictionary for fit instructions
exp = {}                                  
exp["age"] = (0.1, 15.)
exp["tau"] = (0.3, 10.)
exp["massformed"] = (1., 15.)
exp["metallicity"] = (0., 2.5)

dust = {}
dust["type"] = "Calzetti"
dust["Av"] = (0., 2.)

fit_instructions = {}
fit_instructions["redshift"] = (0., 10.)
fit_instructions["exponential"] = exp   
fit_instructions["dust"] = dust

phot = load_goodss(1)  # or 2 or 3
print(type(phot))
print(phot.shape)
print(phot)



<class 'numpy.ndarray'>
(15, 2)
[[8.805800e-01 4.402900e-02]
 [1.572470e+00 7.862350e-02]
 [6.561540e+00 3.280770e-01]
 [1.567080e+01 7.835400e-01]
 [2.316310e+01 1.158155e+00]
 [0.000000e+00 9.900000e+99]
 [0.000000e+00 9.900000e+99]
 [3.790730e+01 1.895365e+00]
 [5.483620e+01 2.741810e+00]
 [0.000000e+00 9.900000e+99]
 [0.000000e+00 9.900000e+99]
 [7.636190e+01 7.636190e+00]
 [5.892980e+01 5.892980e+00]
 [4.955860e+01 4.955860e+00]
 [3.596970e+01 3.596970e+00]]


In [10]:
IDs = np.arange(1, 4, dtype=int)

fit_cat = pipes.fit_catalogue(IDs, fit_instructions, load_goodss, spectrum_exists=False,
                              cat_filt_list=goodss_filt_list, run="guo_cat")

fit_cat.fit(verbose=False)

# BAGPIPES fitting for my mini catalgoue

In [6]:
#trying this but for my mini cat of n galaxies

import numpy as np 
import bagpipes as pipes
from astropy.table import Table
from astropy.io import fits

#load the filter curves
def load_goodss(ID):
    """ Load catalogue. """

    # load up the relevant columns from the catalogue.
    cat = Table.read("mini_cat_20galaxiesonly.fits") 

    
    # Find the correct row for the object we want.
    row = int(ID) - 1

    filters = Table.read('wavelengths_filters.csv', format='csv')
    # Extract filter names
    filters_list = filters['flux_filter'].tolist()

    # Build column names for fluxes and flux errors
    flux_columns = [f'FLUX_APER_{f}_aper_corr_Jy' for f in filters_list]
    fluxerr_columns = [f'FLUXERR_APER_{f}_loc_depth_10pc_Jy' for f in filters_list]
     # Extract fluxes and errors for this galaxy from the catalogue
    fluxes = np.array([cat[row][col] for col in flux_columns])
    fluxes = np.where(fluxes > 0, fluxes, np.nan)
    fluxerrs = np.array([cat[row][col] if not np.ma.is_masked(cat[row][col]) else np.nan for col in fluxerr_columns])


    # Turn these into a 2D array.
    photometry = np.c_[fluxes, fluxerrs]

    # blow up the errors associated with any missing fluxes.
    for i in range(len(photometry)):
        if (photometry[i, 0] == 0.) or (photometry[i, 1] <= 0) or np.isnan(photometry[i, 0]) or np.isnan(photometry[i, 1]):
            photometry[i,:] = [0., 9.9*10**99.]
            
    # Enforce a maximum SNR of 20, or 10 in the IRAC channels.
    for i in range(len(photometry)):
        if i < 10:
            max_snr = 20.
            
        else:
            max_snr = 10.
        
        if photometry[i, 0]/photometry[i, 1] > max_snr:
            photometry[i, 1] = photometry[i, 0]/max_snr

    return photometry

goodss_filt_list = np.loadtxt("filters1/goodss_filt_list1.txt", dtype="str")

#dictionary for fit instructions
exp = {}                                  
exp["age"] = (0.1, 15.)
exp["tau"] = (0.3, 10.)
exp["massformed"] = (1., 15.)
exp["metallicity"] = (0., 2.5)

# lognorm = {}                       # lognormal SFH
# lognorm["tmax"] = (0.01, 15) # these will default to a flat prior probably
# lognorm['fwhm'] = (0.01, 15) 
# lognorm["massformed"] = (5., 12.)  # Log_10 total stellar mass formed: M_Solar

dust = {}
dust["type"] = "Calzetti"
dust["Av"] = (0., 2.)

fit_instructions = {}
fit_instructions["redshift"] = (0., 10.)
fit_instructions["exponential"] = exp
fit_instructions["dust"] = dust


In [7]:
IDs = np.arange(1, 21, dtype=int)

fit_cat = pipes.fit_catalogue(IDs, fit_instructions, load_goodss, spectrum_exists=False,
                              cat_filt_list=goodss_filt_list, run="mini_cat_20_gals_lognormal")

fit_cat.fit(verbose=False)


Bagpipes: fitting object 1



NameError: name 'pmn' is not defined